In [6]:
import psycopg2
from psycopg2 import OperationalError
import pandas as pd

def create_connection(db_name, db_user, db_password, db_host, db_port):
    connection = None
    try:
        connection = psycopg2.connect(
            database=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port,
        )
        print("Connection to PostgreSQL DB successful")
    except OperationalError as e:
        print(f"The error '{e}' occurred")
    return connection


#heroku connection
connection = create_connection(
    "dcegl8mv856qb8", "ndvqpnrwxtmwvu", "eec515b7f7a6c5c44d4df10499aa344d698310c1b39474bd2aefca27633fb241", "ec2-3-89-214-80.compute-1.amazonaws.com", "5432"
)

    
#show all tables in db 
cursor = connection.cursor()
cursor.execute("select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';")
cursor.fetchall()

# #specify table name

df_covid = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/latest/owid-covid-latest.csv')

table_name = "covid"

try:
    cursor.execute(f"DROP TABLE IF EXISTS {table_name};")
    create_table_query = f"CREATE TABLE {table_name} ("
    create_table_query += ", ".join([col + (" double precision" if df_covid.dtypes[i] == 'float' else " varchar(60)") for i, col in enumerate(df_covid.columns)]) + ");"
    cursor.execute(create_table_query)

except Exception as e:
    print(e)
    
for i in range(df_covid.shape[0]):
    row_list = df_covid.iloc[i].tolist()
    str_row = [str(f) for f in row_list]
    str_row = ["NULL" if s == 'nan' else s for s in str_row]
    insert_p1 = f"INSERT INTO {table_name} VALUES ({', '.join(['%s' for i in df_covid.columns])})"
    try:
#         cursor.execute(insert_query)
        cursor.execute(insert_p1, df_covid.iloc[i].tolist())
       
    except Exception as e:
        print(e)
    

cursor.execute("SELECT * FROM covid")
cursor.fetchall()



Connection to PostgreSQL DB successful


[('AFG',
  'Asia',
  'Afghanistan',
  '2021-12-10',
  157858.0,
  273.0,
  63.714,
  7322.0,
  1.0,
  1.571,
  3962.754,
  6.853,
  1.599,
  183.806,
  0.025,
  0.039,
  0.96,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  'NaN',
  5228706.0,
  4397449.0,
  3566192.0,
  nan,
  nan,
  15991.0,
  13.13,
  11.04,
  8.95,
  nan,
  401.0,
  16001.0,
  0.04,
  27.78,
  39835428.0,
  54.422,
  18.6,
  2.581,
  1.337,
  1803.987,
  nan,
  597.029,
  9.59,
  nan,
  nan,
  37.746,
  0.5,
  64.83,
  0.511,
  nan,
  nan,
  nan,
  nan),
 ('OWID_AFR',
  'NaN',
  'Africa',
  '2021-12-10',
  8863302.0,
  27248.0,
  22817.714,
  224145.0,
  113.0,
  119.0,
  6453.141,
  19.839,
  16.613,
  163.194,
  0.082,
  0.087,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  'NaN',
  263737304.0,
  168450910.0,
  109113015.0,
  593495.0,
  367894.0,
  2146925.0,
  19.2,
  12.2

In [ ]:
# cursor.execute("SELECT * FROM covid WHERE location='Afghanistan'")
# cursor.execute("SELECT * FROM covid WHERE new_cases > 10")
# cursor.fetchall()
